In [1]:
import requests
import json
import csv
from bs4 import BeautifulSoup
import os


In [5]:
import pandas as pd
import numpy as np

a=pd.read_csv('data/dict_links.csv',index_col=0)
dict_links=a.to_dict(orient='dict')
dict_links=dict_links['links']

In [2]:
import os
import numpy as np
import pandas as pd

def create_df():
    df=False
    i=0
    for file in os.listdir('data/csv'):
#         print(file, 'parsed')
        if type(df)==bool:
            df=pd.read_csv(f'data/csv/{file}', sep='*')
        else:
            df=df.append(pd.read_csv(f'data/csv/{file}', sep='*'))
        i+=1
    print(f'{i} files parsed!')
    return df
df=create_df()    

161 files parsed!


In [3]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5418 entries, 0 to 11
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   fondo                5418 non-null   object 
 1   name                 5418 non-null   object 
 2   nif                  5418 non-null   int64  
 3   period               5418 non-null   object 
 4   start_date           5418 non-null   object 
 5   end_date             5418 non-null   object 
 6   registro_CNMV        5418 non-null   int64  
 7   email_gest           5405 non-null   object 
 8   rating_depos         5325 non-null   object 
 9   riesgo               5418 non-null   object 
 10  rotacion             4933 non-null   float64
 11  rentab_avg           4933 non-null   float64
 12  n_participaciones    5079 non-null   object 
 13  n_participes         5079 non-null   object 
 14  beneficio            5079 non-null   object 
 15  patrimonio           5079 non-null   obj

#Estrategia:
- rating_depos y  email_gest: imputer "N/A" o n/d
- riesgo???  ver con value_counts pensar en agrupar en bajo/medio/alto?
- comisiones: imputer =0 una vez que tenga todos bien scrapeados
- rentab IIC, volat, ratio.... imputer valor?


In [4]:
df[df['rotacion'].isnull()].fondo.value_counts()

ACCION IBEX 35 ETF, FI COTIZADO ARMONIZADO         46
ACCION EUROSTOXX 50 ETF, FI COTIZADO ARMONIZADO    46
AMUNDI FONDTESORO LARGO PLAZO, FI                  40
AVANCE GLOBAL, FI                                  36
ABANCA RENTA FIJA PATRIMONIO, FI                   31
ABANCA RENTA FIJA FLEXIBLE, FI                     31
ABANCA RENTA VARIABLE MIXTA, FI                    31
ABANCA RENTA VARIABLE EUROPA, FI                   31
ANNUALCYCLES STRATEGIES, FI                        29
ALTAIR PATRIMONIO II, FI                           21
ALTAIR INVERSIONES II, FI                          20
ALTAIR RENTA FIJA DEFENSIVA, FI                    18
ALTAIR EUROPEAN OPPORTUNITIES, FI                  18
ALBUS, FI                                          15
ABANCA GESTION, FI                                 12
ABANCA RENTA FIJA MIXTA, FI                         8
ABANCA RENTA VARIABLE ESPAÑA, FI                    8
ADRIZA RENTA FIJA CORTO PLAZO SOSTENIBLE, FI        6
B&H ACCIONES, FI            

In [32]:
f1=df['fondo']=='ACCION EUROSTOXX 50 ETF, FI COTIZADO ARMONIZADO'
f2=df['rotacion'].isnull()

df[f1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 0 to 45
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   fondo                46 non-null     object 
 1   name                 46 non-null     object 
 2   nif                  46 non-null     int64  
 3   period               46 non-null     object 
 4   start_date           46 non-null     object 
 5   end_date             46 non-null     object 
 6   registro_CNMV        46 non-null     int64  
 7   email_gest           46 non-null     object 
 8   rating_depos         46 non-null     object 
 9   riesgo               46 non-null     object 
 10  rotacion             0 non-null      float64
 11  rentab_avg           0 non-null      float64
 12  n_participaciones    0 non-null      object 
 13  n_participes         0 non-null      object 
 14  beneficio            0 non-null      object 
 15  patrimonio           0 non-null      objec

In [16]:
dict_links

{'1 KESSLER GLOBAL, FI': 'https://www.cnmv.es/portal/Consultas/IIC/Fondo.aspx?nif=V66247677 ',
 'A&G RENTA FIJA CORTO PLAZO, FI': 'https://www.cnmv.es/portal/Consultas/IIC/Fondo.aspx?nif=V85405082 ',
 'ABACO GLOBAL VALUE OPPORTUNITIES FI': 'https://www.cnmv.es/portal/Consultas/IIC/Fondo.aspx?nif=V87152088 ',
 'ABACO RENTA FIJA MIXTA GLOBAL, FI': 'https://www.cnmv.es/portal/Consultas/IIC/Fondo.aspx?nif=V86467461 ',
 'ABANCA GESTION, FI': 'https://www.cnmv.es/portal/Consultas/IIC/Fondo.aspx?nif=V86844974 ',
 'ABANCA RENTA FIJA FLEXIBLE, FI': 'https://www.cnmv.es/portal/Consultas/IIC/Fondo.aspx?nif=V84857739 ',
 'ABANCA RENTA FIJA MIXTA, FI': 'https://www.cnmv.es/portal/Consultas/IIC/Fondo.aspx?nif=V87109906 ',
 'ABANCA RENTA FIJA PATRIMONIO, FI': 'https://www.cnmv.es/portal/Consultas/IIC/Fondo.aspx?nif=V85377406 ',
 'ABANCA RENTA VARIABLE ESPAÑA, FI': 'https://www.cnmv.es/portal/Consultas/IIC/Fondo.aspx?nif=V87112173 ',
 'ABANCA RENTA VARIABLE EUROPA, FI': 'https://www.cnmv.es/portal/Con

In [27]:
fondo='ACCION EUROSTOXX 50 ETF, FI COTIZADO ARMONIZADO'
os.remove(f'data/csv/{fondo}.csv')
print(f'{fondo}.csv deleted')

ACCION EUROSTOXX 50 ETF, FI COTIZADO ARMONIZADO.csv deleted
